In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Load and arrange data**

In [ ]:
!unzip -q '/content/drive/My Drive/IVP_project/archive.zip'

In [ ]:
class_1 = os.listdir('/content/yes/')
class_0 = os.listdir('/content/no/')


In [ ]:
example1 = plt.imread('/content/yes/' + class_1[123])
plt.imshow(example1)
plt.show()
print('Image size=', example1.shape)

example0 = plt.imread('/content/no/' + class_0[54])
plt.imshow(example0)
plt.show()
print('Image size=', example0.shape)


In [ ]:
shutil.rmtree('/content/test/no')

In [ ]:
os.mkdir('/content/train/')
os.mkdir('/content/test/')
os.mkdir('/content/train/yes/')
os.mkdir('/content/train/no/')
os.mkdir('/content/test/yes/')
os.mkdir('/content/test/no/')

In [ ]:
import shutil

np.random.seed(0)
random.seed(0)
train_1, test_1 = train_test_split(class_1, test_size=0.3,  shuffle = True)
train_0, test_0 = train_test_split(class_0, test_size=0.3, shuffle = True)


for filename in train_1:
    shutil.copy('/content/yes/'+filename, '/content/train/yes/'+filename)

for filename in train_0:
    shutil.copy('/content/no/'+filename, '/content/train/no/'+filename)

for filename in test_1:
    shutil.copy('/content/yes/'+filename, '/content/test/yes/'+filename)

for filename in test_0:
    shutil.copy('/content/no/'+filename, '/content/test/no/'+filename)



In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

# **Data generator**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/train/',
    target_size=(227, 227),
    shuffle = True,
    batch_size=32)

validation_generator = test_datagen.flow_from_directory(
    '/content/test/',
    target_size=(227, 227),
    batch_size=32)

Found 176 images belonging to 2 classes.
Found 77 images belonging to 2 classes.


In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.optimizers.SGD(lr=0.0001, momentum=0.9, decay=1e-6), 
              metrics=['accuracy'])
#model.summary()

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
history = model.fit(train_generator,
          epochs=200,
          validation_data=validation_generator)

Epoch 1/200
6/6 [==============================] - 4s 575ms/step - loss: 1.6434 - accuracy: 0.4910 - val_loss: 0.6904 - val_accuracy: 0.6364
Epoch 2/200
6/6 [==============================] - 3s 497ms/step - loss: 1.6038 - accuracy: 0.5290 - val_loss: 0.6912 - val_accuracy: 0.6494
Epoch 3/200
6/6 [==============================] - 3s 496ms/step - loss: 1.1714 - accuracy: 0.5833 - val_loss: 0.6879 - val_accuracy: 0.6104
Epoch 4/200
6/6 [==============================] - 3s 503ms/step - loss: 1.2315 - accuracy: 0.6590 - val_loss: 0.6874 - val_accuracy: 0.6364
Epoch 5/200
6/6 [==============================] - 3s 502ms/step - loss: 1.3941 - accuracy: 0.6530 - val_loss: 0.6845 - val_accuracy: 0.6364
Epoch 6/200
6/6 [==============================] - 3s 496ms/step - loss: 0.8392 - accuracy: 0.7313 - val_loss: 0.6831 - val_accuracy: 0.6104
Epoch 7/200
6/6 [==============================] - 3s 501ms/step - loss: 1.2278 - accuracy: 0.6004 - val_loss: 0.6845 - val_accuracy: 0.6234
Epoch 8/200
6

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# **5-FOLD CV**

# **Test train split and save**

In [ ]:
np.random.seed(0)
random.seed(0)
class_1 = os.listdir('/content/yes/')
class_0 = os.listdir('/content/no/')


In [ ]:
#
##run if you need to set aside a test set only
###
####
train_1, test_1 = train_test_split(class_1, test_size=0.3,  shuffle = True)
train_0, test_0 = train_test_split(class_0, test_size=0.3, shuffle = True)

for filename in train_1:
    shutil.copy('/content/yes/'+filename, '/content/train/'+filename)

for filename in train_0:
    shutil.copy('/content/no/'+filename, '/content/train/'+filename)

for filename in test_1:
    shutil.copy('/content/yes/'+filename, '/content/test/'+filename)

for filename in test_0:
    shutil.copy('/content/no/'+filename, '/content/test/'+filename)


In [ ]:
#
## run for using full set for CV
###
####
for filename in class_1:
    shutil.copy('/content/yes/'+filename, '/content/train/'+filename)

for filename in class_0:
    shutil.copy('/content/no/'+filename, '/content/train/'+filename)

In [ ]:
rm -rf `find -type d -name .ipynb_checkpoints`

# **Create dataframe for test and train**

In [ ]:
#train_files = train_1+train_0
#train_label = np.concatenate((np.ones(len(train_1)), np.zeros((len(train_0)))))

#test_files = test_1+test_0
#test_label = np.concatenate((np.ones(len(test_1)), np.zeros((len(test_0)))))

train_files = class_1 + class_0
train_label = np.concatenate((np.ones(len(class_1)), np.zeros((len(class_0)))))

train_df = pd.DataFrame({
    'Filename': train_files,
    'Label': train_label
})

#test_df = pd.DataFrame({
#    'Filename': test_files,
#    'Label': test_label
#})

train_df.to_csv('/content/trainlabel.csv')
#test_df.to_csv('/content/testlabel.csv')

# **Model**

In [ ]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
def alexnet():
  return keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='softmax')
])


# **Data generator**

## **Cross  validation**

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
train_data = pd.read_csv('/content/trainlabel.csv')
Y = train_data[['Label']]
train_data = train_data.replace([1,0], ['yes', 'no'])
train_data = train_data.sample(frac=1).reset_index(drop=True)
n = len(Y)
kf = KFold(n_splits = 5)
                         
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

In [ ]:
print(train_data.head(3))
print(train_data.tail(3))

  Unnamed: 0   Filename Label
0        158  14 no.jpg    no
1         83    Y61.jpg   yes
2        170  35 no.jpg    no
    Unnamed: 0  Filename Label
250        117   Y14.jpg   yes
251         47  Y169.jpg   yes
252        172   N21.jpg    no


In [ ]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = '/saved_models/'
fold_var = 1
image_dir = '/content/train/'
num_epochs = 400

for train_index, val_index in skf.split(np.zeros(n),Y):
	training_data = train_data.iloc[train_index]
	validation_data = train_data.iloc[val_index]
	
	train_data_generator = train_datagen.flow_from_dataframe(training_data, directory = image_dir,
						       x_col = "Filename", y_col = "Label",
						       class_mode = "categorical", shuffle = True)
	valid_data_generator  = test_datagen.flow_from_dataframe(validation_data, directory = image_dir,
							x_col = "Filename", y_col = "Label",
							class_mode = "categorical", shuffle = True)
	
	# CREATE NEW MODEL
	model = alexnet()
	# COMPILE NEW MODEL
	model.compile(loss='categorical_crossentropy',
		      optimizer=tf.optimizers.SGD(lr=0.00001, momentum=0.9, decay=1e-6),
		      metrics=['accuracy'])
	
	# CREATE CALLBACKS
	checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
	callbacks_list = [checkpoint]
	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
	history = model.fit(train_data_generator,
			    epochs=num_epochs,
			    callbacks=callbacks_list,
			    validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
	model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_LOSS.append(results['loss'])
	
	tf.keras.backend.clear_session()
	
	fold_var += 1

Streaming output truncated to the last 5000 lines.

Epoch 00353: val_accuracy did not improve from 0.96078
Epoch 354/400
7/7 [==============================] - 4s 506ms/step - loss: 0.2266 - accuracy: 0.9327 - val_loss: 0.2355 - val_accuracy: 0.8627

Epoch 00354: val_accuracy did not improve from 0.96078
Epoch 355/400
7/7 [==============================] - 4s 572ms/step - loss: 0.5436 - accuracy: 0.7819 - val_loss: 0.2715 - val_accuracy: 0.8431

Epoch 00355: val_accuracy did not improve from 0.96078
Epoch 356/400
7/7 [==============================] - 4s 568ms/step - loss: 0.2345 - accuracy: 0.9233 - val_loss: 0.4025 - val_accuracy: 0.7647

Epoch 00356: val_accuracy did not improve from 0.96078
Epoch 357/400
7/7 [==============================] - 4s 510ms/step - loss: 0.2669 - accuracy: 0.8562 - val_loss: 0.4614 - val_accuracy: 0.7647

Epoch 00357: val_accuracy did not improve from 0.96078
Epoch 358/400
7/7 [==============================] - 4s 516ms/step - loss: 0.3456 - accuracy: 0.8

In [ ]:
VALIDATION_ACCURACY

[0.8627451062202454,
 0.9607843160629272,
 0.843137264251709,
 0.8600000143051147,
 0.9599999785423279]

In [ ]:
VALIDATION_LOSS 

[0.7199138402938843,
 0.22379471361637115,
 0.6414989233016968,
 0.4350334405899048,
 0.323379248380661]

In [ ]:
shutil.copy('/saved_models/model_4.h5', '/content/drive/MyDrive/IVP_project/alexnet_models/')

'/content/drive/MyDrive/IVP_project/alexnet_models/model_4.h5'